In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import os
from processingmm import batch_processing
from histopolalign import combine_images, prepare_images, semi_automatic_tool, align_folders, match_skeletons, align_imgs, get_numerical_results

# 1. Process the measurements that will be aligned

In [2]:
# get the folder in which the polarimetric measurements are stored
neoplastic_polarimetry_path = os.path.join(os.getcwd().split('notebooks')[0], 'data', 'TumorMeasurements')
neoplastic_folder = '21'
polarimetry_path = os.path.join(neoplastic_polarimetry_path, neoplastic_folder)
directories = [os.path.join(neoplastic_polarimetry_path, neoplastic_folder)]

In [3]:
# process the measurements, if needed
calib_directory = os.path.join(os.getcwd().split('notebooks')[0], 'calib')
batch_processing.batch_process(directories, calib_directory, run_all = False)

Traceback (most recent call last):
  File "C:\Users\romai\Documents\processingMM\src\processingmm\batch_processing.py", line 70, in find_folder_name
    assert len(re.findall("[\d]{4}-[\d]{2}-[\d]{2}", root)) == 1
AssertionError
0it [00:00, ?it/s]


[]

# 2. Get the combined masks for the histology folders

In [8]:
# create the folde that will be used to align pathology and polarimetry
combine_images.create_to_align_folder()

# load the pathology images...
histology_path = os.path.join(os.getcwd().split('notebooks')[0], 'data', 'HistologyResults')
imgs_all = combine_images.load_the_images(histology_path, Verbose = True)

# ...and process them
combine_images.get_combined_img(imgs_all, force_recompute = True, Verbose = True)

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:02<00:00, 14.71it/s]


Load the images: 2.261 seconds.
Get combined images: 0.384 seconds.


# 3. Obtain the parameters (manually using the GUI) to first align the histology and the polarimetry

### 3.1. Prepare the images

In [13]:
alignment_measurements = prepare_images.create_the_alignments(histology_path, polarimetry_path, Verbose = True)
alignment_measurements

Get polarimetry filenames and rename images: 0.001 seconds.
Create the alignment objects: 0.004 seconds.


[Folder path: C:\Users\romai\Documents\PathologyPaper\data\TumorMeasurements\21\2022-06-29_T_HORAO-21-AF_FR_S2_1, histology path : C:\Users\romai\Documents\PathologyPaper\data\HistologyResults\21\HORAO-21-S2-3,  status: initialized,
 Folder path: C:\Users\romai\Documents\PathologyPaper\data\TumorMeasurements\21\2022-06-29_T_HORAO-21-BF_FR_S2_1, histology path : C:\Users\romai\Documents\PathologyPaper\data\HistologyResults\21\HORAO-21-S2-1,  status: initialized]

### 3.2. Load the polarimetry, labels and histology (H&E) images for each polarimetry folder

In [14]:
alignment_measurements = prepare_images.load_and_preprocess_imgs(alignment_measurements, force_recompute = False, Verbose = True)

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Get the images and center of mass: 1.443 seconds.


 50%|██████████████████████████████████████████                                          | 1/2 [00:01<00:01,  1.70s/it]

Load the images: 0.260 seconds.
Get the images and center of mass: 1.199 seconds.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.56s/it]

Load the images: 0.214 seconds.


### 3.3. Semi-automatic rotation/alignement tool

In [15]:
for measurement in alignment_measurements:
    selector = semi_automatic_tool.ask_for_parameters(measurement, force_recompute = True)

# 4. Actually perform the alignment

In [16]:
align_folders.create_align_folders(alignment_measurements, Verbose = True)

 50%|██████████████████████████████████████████                                          | 1/2 [00:00<00:00,  2.23it/s]

Process the images to apply the selected changes: 0.311 seconds.
Save the processed images: 0.042 seconds.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.95it/s]

Process the images to apply the selected changes: 0.462 seconds.
Save the processed images: 0.037 seconds.


### 4.1. Automatic part

In [17]:
border_parameter = 5
nsamples = 400
max_distance = 150

for measurement in tqdm(alignment_measurements):
    match_skeletons.match_skeletons(measurement, border_parameter, nsamples = nsamples, max_distance = max_distance,
                                                                   Verbose = True)

 50%|██████████████████████████████████████████                                          | 1/2 [00:16<00:16, 16.83s/it]

Match the two skeletons: 16.825 seconds.


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:32<00:00, 16.15s/it]

Match the two skeletons: 15.477 seconds.


### 4.2. Semi-automatic part

In [19]:
align_folders.semi_automatic_processing(alignment_measurements)

In [20]:
alignment_measurements = align_imgs.align_img_master(alignment_measurements)

### 4.3. Get the final masks and overlay them on the polarimetric parameter maps

In [21]:
align_imgs.generate_final_masks(alignment_measurements, Verbose = False)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:31<00:00, 15.57s/it]


### 4.4. Finalize and save the numerical results

In [22]:
values = get_numerical_results.create_numerical_values(alignment_measurements)